In this notebook, we will use regression algorithms to predict the number of rented bikes for a given day

### Load data

In [ ]:
rental = pd.read_csv('data/rental.csv')

# Predictive modeling

Standardization function is copied from PS4:

In [13]:
import numpy as np

def standardize(raw_data):
    return ((raw_data - np.mean(raw_data, axis = 0)) / np.std(raw_data, axis = 0))

In [14]:
from sklearn.model_selection import train_test_split

X = data.loc[:, data.columns != 'cnt']
y = data['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X.head()

,instant,temp,atemp,hum,windspeed,season_1,season_2,season_3,season_4,year_0,...,weekday_3,weekday_4,weekday_5,weekday_6,workingday_0,workingday_1,weathersit_1,weathersit_2,weathersit_3,weathersit_4
0,1,0.24,0.2879,0.81,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
1,2,0.22,0.2727,0.80,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,3,0.22,0.2727,0.80,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
3,4,0.24,0.2879,0.75,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
4,5,0.24,0.2879,0.75,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0


## Method 1: Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
score = log_reg.score(X_test, y_test)
print(score)

0.014672036823935558


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Method 2: Multiple Linear Regression

In [41]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
score = lin_reg.score(X_train, y_train)
print(f'Predictive score on training data {score:.4f}')

score = lin_reg.score(X_test, y_test)
print(f'Predictive score on test data {score:.4f}')

Predictive score on training data 0.6896
Predictive score on test data 0.6719


## Method 3: K-Nearest Neighbors

In [42]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
pred = knn.predict(X_test)
train_score = knn.score(X_train, y_train)
print(f'Predictive score on training data {train_score:.4f}')
test_score = knn.score(X_test, y_test)
print(f'Predictive score on test data {test_score:.4f}')

Predictive score on training data 0.1318
Predictive score on test data 0.0158


In [43]:
from sklearn.model_selection import GridSearchCV

knn_tuned = KNeighborsClassifier()
param_grid = {'n_neighbors':range(1,15), 'p':range(1,3)}
grid_search = GridSearchCV(knn_tuned, param_grid, return_train_score=True)
grid_search.fit(X_train, y_train)
test_accuracy = grid_search.cv_results_['mean_test_score'].mean()
train_accuracy = grid_search.cv_results_['mean_train_score'].mean()

print(f'Predictive score on training data {train_accuracy:.4f}')
print(f'Predictive score on test data {test_accuracy:.4f}')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Predictive score on training data 0.2115
Predictive score on test data 0.0125


## Method 4. Gradient Boosting

In [48]:
from sklearn.ensemble import GradientBoostingRegressor

param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [2, 5, 6, 7, 9, 11],
    'n_estimators': [10, 50, 100],
    'min_samples_leaf': [1, 3, 5, 7, 9]
}

gbm = GradientBoostingRegressor(random_state=seed)

cv_search = GridSearchCV(gbm, param_grid, cv=5, verbose=True, n_jobs=-1).fit(X_train, y_train)
print("best parameters:", cv_search.best_params_)
# best parameters: {'learning_rate': 0.1, 'max_depth': 11, 'min_samples_leaf': 7, 'n_estimators': 100}

Fitting 5 folds for each of 270 candidates, totalling 1350 fits
best parameters: {'learning_rate': 0.1, 'max_depth': 11, 'min_samples_leaf': 7, 'n_estimators': 100}


In [49]:
print('Training set {:.4f}'.format(cv_search.best_estimator_.fit(X_train, y_train).score(X_train, y_train)))
print('Cross-validated set: {:.4f}'.format(cv_search.best_score_))
print('Test set: {:.4f}'.format(cv_search.best_estimator_.score(X_test, y_test)))

Training accuracy score 0.9865
Cross-validated accuracy score: 0.9387
Test performance: 0.9440


## Method 5. Decision Trees

## Method 6. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor


## Method 7. Support Vector Machine

## Method 8. Neural Networks